# 1. Recol·lecció i tractament de dades

 * 1.1 Executem la llibreria LabelMe, que manualment ens permetrà obtenir unes coordenades que marquin la àrea del objecte que volem detectar. Aquesta llibreria ens retornarà un arxiu .json amb els valors numèrics dels punts marcats. Amb aquests valors, la xarxa neuronal serà capaç d'entrenar-se. Haurem de crear una carpeta que anomenarem "labels" on hi aniran aquestes "etiquetes" en format .json

In [ ]:
!labelme #execució de la llibreria labelme

# 2. Creació de la base de dades

In [3]:
#importació de llibreries necessàries
import tensorflow as tf
import json
import numpy as np
from matplotlib import pyplot as plt
import os

In [ ]:
#limitació de la memòria GPU 
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
imatges = tf.data.Dataset.list_files('data\\images\\*.png') # introduïm les imatges a la base de dades

In [ ]:
def carregar_imatge(imatge): #funció per a carregar imatges
    byte_img = tf.io.read_file(imatge)
    img = tf.io.decode_png(byte_img)
    return img

In [ ]:
imatges = imatges.map(carregar_imatge) # executem la funció per carregar les imatges en la base de dades 

In [ ]:
imatges.as_numpy_iterator().next() 

In [ ]:
def quantImatgesDividir(dir_carpeta):
    num_elements = len(os.listdir(dir_carpeta))
    n_entrenar = round(num_elements*0.7) # 70% de les imatges per entrenar
    n_provar = round(num_elements*0.15) # 15% de les imatges per provar
    n_validar = num_elements - (n_entrenar + n_provar) # 15% de les imatges per validar
    print(f"{n_entrenar} --> 'train' \n {n_provar} --> 'test' \n {n_validar} --> 'val'")


In [ ]:
quantImatgesDividir('T:\REP\deteccio_prova\data\images') # s'han de dividir manualment les imatges (intentant que sigui aleatori)

In [ ]:
# mou les els labels a les seves respectives carpetes
for carpeta in ['train','test','val']:
    for arxiu in os.listdir(os.path.join('data', carpeta, 'images')): #per cada arxiu en cada carpeta
        nom_arxiu = arxiu.split('.')[0]+'.json' #extreu el nom de l'arxiu
        dir_existent = os.path.join('data','labels', nom_arxiu)#crea possible direcció
        if os.path.exists(dir_existent): #si la possible direcció existeix, desplaça l'arxiu .json corresponent
            nova_dir = os.path.join('data',carpeta,'labels',nom_arxiu)
            os.replace(dir_existent, nova_dir)#canviem

### 2.1 Augmentem la quantitat de dades amb la llibreria Albumentation

In [4]:
import albumentations as alb
import cv2

In [ ]:
augmentor = alb.Compose([alb.RandomCrop(width=1024, height=1024),
                         alb.HorizontalFlip(p=0.5), 
                         alb.RandomBrightnessContrast(p=0.2),
                         alb.RandomGamma(p=0.2), 
                         alb.RGBShift(p=0.2), 
                         alb.VerticalFlip(p=0.5)], 
                       bbox_params=alb.BboxParams(format='albumentations', #format de coords que volem (buscar docs)
                                                  label_fields=['class_labels']))

In [ ]:
for partition in ['train','test','val']: 
    for image in os.listdir(os.path.join('data', partition, 'images')):
        img = cv2.imread(os.path.join('data', partition, 'images', image))

        coords = [0,0,0.00001,0.00001]
        label_path = os.path.join('data', partition, 'labels', f'{image.split(".")[0]}.json')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label = json.load(f)

            coords[0] = label['shapes'][0]['points'][0][0]
            coords[1] = label['shapes'][0]['points'][0][1]
            coords[2] = label['shapes'][0]['points'][1][0]
            coords[3] = label['shapes'][0]['points'][1][1]
            coords = list(np.divide(coords, [1024,1024,1024,1024]))
            # ^^^ carregar imatges i json ^^^

        try: 
            for x in range(60):# quantitat d'imatges que surten d'una imatge base
                augmented = augmentor(image=img, bboxes=[coords], class_labels=['Maduixa'])
                cv2.imwrite(os.path.join('aug_data', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'),augmented['image'])
                
                annotation = {}
                annotation['image'] = image

                if os.path.exists(label_path):
                    if len(augmented['bboxes']) == 0: 
                        annotation['bbox'] = [0,0,0,0]
                        annotation['class'] = 0 
                    else: 
                        annotation['bbox'] = augmented['bboxes'][0]
                        annotation['class'] = 1
                else: 
                    annotation['bbox'] = [0,0,0,0]
                    annotation['class'] = 0 
                    '''
                     La variable “annotation” és un diccionari que conté informació sobre la imatge i 
                     la seva anotació. Si el fitxer d’anotació existeix, el codi afegeix la caixa delimitadora
                     i la classe corresponent a l’objecte detectat a l’estructura “annotation”. 
                     Si el fitxer d’anotació no existeix, el codi assigna una caixa delimitadora i 
                     una classe buida a l’estructura “annotation”.
                    '''
                with open(os.path.join('aug_data', partition, 'labels', f'{image.split(".")[0]}.{x}.json'), 'w') as f:
                    json.dump(annotation, f)

        except Exception as e:
            print(e)

### 2.2 Incloure imatges creades amb Albumentations a la Dataset

In [ ]:
#Obrim les carpetes amb les img a manipular i les carreguem
train_images = tf.data.Dataset.list_files('aug_data\\train\\images\\*.jpg',shuffle=False)
train_images = train_images.map(carregar_imatge)
#Es fa resize de les img i també baixem l'escala de la imatge a 1
train_images = train_images.map(lambda x: tf.image.resize(x, (250,250)))
train_images = train_images.map(lambda x: x/255)

In [ ]:
test_images = tf.data.Dataset.list_files('aug_data\\test\\images\\*.jpg', shuffle=False)
test_images = test_images.map(carregar_imatge)
test_images = test_images.map(lambda x: tf.image.resize(x, (250,250)))
test_images = test_images.map(lambda x: x/255)

In [ ]:
val_images = tf.data.Dataset.list_files('aug_data\\val\\images\\*.jpg', shuffle=False)
val_images = val_images.map(carregar_imatge)
val_images = val_images.map(lambda x: tf.image.resize(x, (250,250)))
val_images = val_images.map(lambda x: x/255)

### 2.3 Carregar Labels

In [ ]:
#funció que carrega els labels
def carregar_labels(x):
    with open(x.numpy(), 'r', encoding='utf-8') as f:
        label = json.load(f)
    return [label['class']],label['bbox']

In [ ]:
train_labels = tf.data.Dataset.list_files('aug_data\\train\\labels\\*.json', shuffle=False)
train_labels = train_labels.map(lambda x: tf.py_function(carregar_labels, [x], [tf.uint8, tf.float16]))#tf_py_function(funció, paràmetres, tipus de retorn) 
#obre els labels i els aplica la funció carregar_labels

In [ ]:
test_labels = tf.data.Dataset.list_files('aug_data\\test\\labels\\*.json', shuffle=False)
test_labels = test_labels.map(lambda x: tf.py_function(carregar_labels, [x], [tf.uint8, tf.float16]))

In [ ]:
val_labels = tf.data.Dataset.list_files('aug_data\\val\\labels\\*.json', shuffle=False)
val_labels = val_labels.map(lambda x: tf.py_function(carregar_labels, [x], [tf.uint8, tf.float16]))

### 2.4 Combinar labels i imatges

In [ ]:
#cal comprovar quantes imatges hi ha en cada carpeta
print("Train: " + str(len(train_labels)) + ", Test: " + str(len(test_labels)) + ", Val: " + str(len(val_labels)))

In [ ]:
train = tf.data.Dataset.zip((train_images, train_labels)) #mètode que junta les imatges amb els labels
train = train.shuffle(2000) #posar el número que hi ha apoximat cap amunt, en aquest cas 2000
train = train.batch(8)#crear lots de 8
train = train.prefetch(4)#redueix la capacitat de procesar per evitar errors

In [ ]:
test = tf.data.Dataset.zip((test_images, test_labels))
test = test.shuffle(500)
test = test.batch(8)
test = test.prefetch(4)

In [ ]:
val = tf.data.Dataset.zip((val_images, val_labels))
val = val.shuffle(500)
val = val.batch(8)
val = val.prefetch(4)

In [ ]:
train.as_numpy_iterator().next()[1]
'''
Amb això visualitzem la manera en què hem codificat i guardat les imatges;
Obtindrem això:
(array([[n],
        [n],
        [n],
        [n],
        [n],
        [n],
        [n],
        [n]], dtype=uint8),
 array([[n , n , n , n],
        [n , n , n , n],
        [n , n , n , n],
        [n , n , n , n],
        [n , n , n , n],
        [n , n , n , n],
        [n , n , n , n],
        [n , n , n , n]], dtype=float16))

la primera columna ens indica la classe de la imatge, és a dir, l'objecte que hi ha a la imatge
la segona columna ens indica la bounding box de la imatge, és a dir, la posició de l'objecte a la imatge

'''

# 3. Creació de la IA

In [ ]:
#La línia 2 importa les capes de Input, de convolució 2D, de connexió i de agregació màxima global
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16

In [ ]:
vgg = VGG16(include_top=False) 
#Marca que les ultimes capes de la xarxa no les utilitzarem perque afegirem les nostrtes propies!

In [ ]:
vgg.summary()

In [ ]:
'''
La funció crea i retorna un model de xarxa neuronal convolucional per a la detecció dels objectes. 
El model té dues parts: una per a determinar si una imatge té una maduixa o no i una altra per a localitzar la maduixa en la imatge. 
Al utilitzar VGG16, com que esta pre-entrenada, les característiques del INPUT
i les classifica en dues branques que son les dues parts mencionades abans de manera "automàtica". 
'''

def build_model(): 
    input_layer = Input(shape=(250,250,3))
    
    vgg = VGG16(include_top=False)(input_layer)
    #Model de classificació
    f1 = GlobalMaxPooling2D()(vgg)
    class1 = Dense(2048, activation='relu')(f1) #relu == funció que determina la classe; si és maduixa o no
    class2 = Dense(1, activation='sigmoid')(class1) # sigmoid == funció que determina la probabilitat
    '''
    sigmoid = f(x) = 1/(1+e^-x)
    '''
    #Model de localització de coordenades
    f2 = GlobalMaxPooling2D()(vgg)
    regress1 = Dense(2048, activation='relu')(f2)
    regress2 = Dense(4, activation='sigmoid')(regress1)
    
    detector = Model(inputs=input_layer, outputs=[class2, regress2])
    return detector

In [ ]:
train.as_numpy_iterator().next()[1]

### 3.1 Prova de la xarxa neuronal

In [ ]:
detector = build_model()

In [ ]:
detector.summary()
'''
Aquest fragment dona una descripció resumida de la xarxa que acabem de construir,
inclou el nombre de capes, el nombre de paràmetres que ha après durant un entrenament i 
la forma de les dades que passen entre les capes.
'''

In [ ]:
X, y = train.as_numpy_iterator().next()

In [ ]:
X.shape

In [ ]:
classes, coords = detector.predict(X)

In [ ]:
classes, coords

### 3.2 Definir pèrdues i optimitzadors
Els optimitzadors són algorismes que s’utilitzen per ajustar els pesos d’una xarxa neuronal durant l’entrenament. Els optimitzadors són responsables de minimitzar la funció de pèrdua de la xarxa neuronal.

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001) #li introduïm el decay que hem calculat a l'optimitzador

'''
En el context de les xarxes neuronals, un optimitzador és un algorisme
que ajuda a ajustar els paràmetres de la xarxa per aconseguir una millor precisió.
L'optimitzador Adam és un exemple d'això i ajuda a l'optimitzador a convergir més ràpidament
i amb més precisió.
'''

La funció té dos components: la pèrdua de localització i la pèrdua de classificació.
La pèrdua de localització mesura la diferència entre les coordenades dels quadres delimitadors 
predits i les coordenades dels quadres delimitadors reals. La pèrdua de classificació mesura la
diferència entre les probabilitats de classe predites i les probabilitats de classe reals. 
En aquesta funció, només es calcula la pèrdua de localització.

La funció té com a entrada dos tensors: y_true i yhat. 
y_true conté les coordenades dels quadres delimitadors reals i
les probabilitats de classe reals per a cada objecte en la imatge d’entrada.
yhat conté les coordenades dels quadres delimitadors predits i les
probabilitats de classe predites per a cada objecte en la imatge d’entrada.

La funció calcula la pèrdua de localització sumant el quadrat de la diferència
entre les coordenades dels quadres delimitadors reals i les coordenades dels quadres
delimitadors predits. A continuació, calcula la diferència entre l’amplada i l’alçada
dels quadres delimitadors reals i els quadres delimitadors predits i suma els quadrats
d’aquestes diferències. Finalment, retorna la suma de les dues pèrdues.

In [ ]:
def localization_loss(y_true, yhat):#primer valor: coordenades reals, segon valor: coordenades previstes     
    delta_coord = tf.reduce_sum(tf.square(y_true[:,:2] - yhat[:,:2])) #diferència dels dos primers valors de cada fila de la matriu
                  
    h_true = y_true[:,3] - y_true[:,1] #quarta columna d'una matriu - segona columna
    w_true = y_true[:,2] - y_true[:,0] #tercera columna - primera

    h_pred = yhat[:,3] - yhat[:,1] 
    w_pred = yhat[:,2] - yhat[:,0] 
    '''
    delta_size = suma dels quadrats de les diferències entre les dimensions originals 
    i les dimensions reconstruïdes de l'imatge.
    '''
    delta_size = tf.reduce_sum(tf.square(w_true - w_pred) + tf.square(h_true-h_pred))
    return delta_coord + delta_size

In [ ]:
classloss = tf.keras.losses.BinaryCrossentropy() #model que fa una classificació binaria 
regressloss = localization_loss #model que acabem de crear

In [ ]:
#test, en les 2 cel·les inferiors s'hauria d'obtenir : <tf.Tensor: shape=(), dtype=float32, numpy=n>
regressloss(y[1], coords) #y[0] ==> 0 o 1 (hi ha maduixa o no) , y[1] ==> coords (tot del batch sencer)

In [ ]:
classloss(y[0], classes) #numpy ==> probabilitat

In [ ]:
localization_loss(y[1],coords)